<a href="https://colab.research.google.com/github/AndersonxxAlves/Projeto_gravacao/blob/main/projeto_gravacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
language = 'pt'

In [ ]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode
import os


GRAVACAO = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const recorder = async (duration) => {
  try {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    const chunks = [];

    mediaRecorder.ondataavailable = e => chunks.push(e.data);
    mediaRecorder.start();

    await sleep(duration);

    mediaRecorder.stop();

    return new Promise(resolve => {
      mediaRecorder.onstop = async () => {
        const blob = new Blob(chunks, { type: 'audio/wav' });
        const reader = new FileReader();
        reader.onloadend = () => resolve(reader.result);
        reader.readAsDataURL(blob);
      };
    });
  } catch (error) {
    console.error('Erro ao gravar:', error);
    throw error;
  }
};
"""

def gravacao(sec=5):

    gravacao_entrada = output.eval_js(GRAVACAO + '; recorder(%s)' % (sec * 1000))
    audio = b64decode(gravacao_entrada.split(',')[1])
    nome_arquivo = 'audio_teste.wav'
    with open(nome_arquivo, 'wb') as f:
        f.write(audio)
    return f'/content/{nome_arquivo}'

if __name__ == "__main__":
    print('Ouvindo por 5 segundos...')
    arquivo_gravado = gravacao(5)
    print(f'\nÁudio gravado com sucesso!!')

    display(Audio(arquivo_gravado, autoplay=False))

Ouvindo por 5 segundos...

Áudio gravado com sucesso!!


In [ ]:
import whisper
language = 'pt'
print(f"Iniciando a transcrição...")
formato = whisper.load_model('small')
resultado = formato.transcribe(arquivo_gravado, fp16=False, language=language)
transcrever = resultado['text']
print(f"\n{transcrever}")

Iniciando a transcrição...

 Me explique em poucas palavras o que é um carro.


In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key="**"
)

response = client.responses.create(
  model="gpt-5-nano",
  input=transcrever,
  store=True,
)
print(f'Resposta:')
resposta_chat = response.output_text
print(resposta_chat);

Resposta:
Um automóvel é um veículo motorizado de quatro rodas usado para transportar pessoas.


In [ ]:
from gtts import gTTS

gtts_object = gTTS(text=resposta_chat, lang=language, slow=False)
audio_resposta = "/content/audio_resposta.wav"
gtts_object.save(audio_resposta)
print("Convertendo texto em audio...")
display(Audio(audio_resposta, autoplay=True))

Convertendo texto em audio...
